In [70]:
import xarray as xr
# Open netCDF file using xarray
fm = xr.open_dataset("/glade/work/kumar34/prj_data/surf_data/CLM4.0CN.h0.fixed.nc")

# Extract variables using xarray
lm = fm.landmask
area = fm.area
landfrac = fm.landfrac

# Set _FillValue
area.attrs["_FillValue"] = 1.0e36
landfrac.attrs["_FillValue"] = 1.0e36

# Manipulate variables using xarray
arwt = area * landfrac / 100
lat = fm["lat"]
lon = fm["lon"]

In [ ]:
# open NetCDF file
file_path = "/glade/work/kumar34/CESM2-LE-LU/BHIST_SSP370smbb_r3_LU_v3.clm2.h0.TSA.185001-210012.nc"
f1 = xr.open_dataset(file_path)

# extract variable and print summary
var1 = f1.TSA
print(var1)

In [3]:
import numpy as np
import xarray as xr
import glob
import os
import dask

nMod = 100
modName = np.array(["1001.001", "1021.002", "1041.003", "1061.004", "1081.005", "1101.006", "1121.007",
                    "1141.008", "1161.009", "1181.010", "1011.001", "1031.002", "1051.003", "1071.004",
                    "1091.005", "1111.006", "1131.007", "1151.008", "1171.009", "1191.010", "1231.001",
                    "1231.002", "1231.003", "1231.004", "1231.005", "1231.006", "1231.007", "1231.008",
                    "1231.009", "1231.010", "1231.011", "1231.012", "1231.013", "1231.014", "1231.015",
                    "1231.016", "1231.017", "1231.018", "1231.019", "1231.020", "1251.001", "1251.002",
                    "1251.003", "1251.004", "1251.005", "1251.006", "1251.007", "1251.008", "1251.009",
                    "1251.010", "1251.011", "1251.012", "1251.013", "1251.014", "1251.015", "1251.016",
                    "1251.017", "1251.018", "1251.019", "1251.020", "1281.001", "1281.002", "1281.003",
                    "1281.004", "1281.005", "1281.006", "1281.007", "1281.008", "1281.009", "1281.010",
                    "1281.011", "1281.012", "1281.013", "1281.014", "1281.015", "1281.016", "1281.017",
                    "1281.018", "1281.019", "1281.020", "1301.001", "1301.002", "1301.003", "1301.004",
                    "1301.005", "1301.006", "1301.007", "1301.008", "1301.009", "1301.010", "1301.011",
                    "1301.012", "1301.013", "1301.014", "1301.015", "1301.016", "1301.017", "1301.018",
                    "1301.019", "1301.020"])
#print(modName)

In [12]:
inDirT = "/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/TREFHT/"
inDirP = "/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PSL/"
inDirU = "/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/U10/"
inDirRH = "/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/RH2M/"
inDirS = "/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/FSA/"
inDirL = "/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/FIRA/"
inDirSM = "/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/H2OSOI/"
outDir = "/glade/scratch/tkavoo/PET/"

## Calculating PET
Here we shall be dealing with large chunks being produced that might lead to a warning generation. We can ignore the warning as it does not affect the output or if you do not want to see the warning, we can configure the setting to allow for large chunks using the code below.

We will choose to accept the large chunk by setting array.slicing.split_large_chunks to False as suggested by the warning message. You can use the following code to do that:

In [13]:
#To accept the large chunk
#dask.config.set(**{'array.slicing.split_large_chunks': False})

#To split the chunk into smaller ones
dask.config.set(**{'array.slicing.split_large_chunks': True})

# Define number of models to evaluate
nEns = min(2, len(modName))
# Loop over models
for i in range(nEns):
    Tfilename = glob.glob((f"{inDirT}*{modName[i]}*.h0.*.nc")) #f"{inDirT}*.{modDir}*.h0.*nc"
    RHfilename = glob.glob((f"{inDirRH}*{modName[i]}*.h0.*.nc"))#f"{inDirRH}*{modDir}*.h0*.nc"
    Pfilename = glob.glob((f"{inDirP}*{modName[i]}*.h0.*.nc"))#f"{inDirP}*{modDir}*.h0*.nc"
    Sfilename = glob.glob((f"{inDirS}*{modName[i]}*.h0.*.nc"))#f"{inDirS}*{modDir}*.h0*.nc"
    Lfilename = glob.glob((f"{inDirL}*{modName[i]}*.h0.*.nc"))#f"{inDirL}*{modDir}*.h0*.nc"
    Ufilename = glob.glob((f"{inDirU}*{modName[i]}*.h0.*.nc"))#f"{inDirU}*{modDir}*.h0*.nc"
    
    # Open input files
    T = xr.open_mfdataset(Tfilename, decode_times=True, combine='by_coords')#xr.open_dataset(Tfilename)
    RH = xr.open_mfdataset(RHfilename, decode_times=True, combine='by_coords')#xr.open_dataset(RHfilename)
    P = xr.open_mfdataset(Pfilename, decode_times=True, combine='by_coords')#xr.open_dataset(Pfilename)
    S = xr.open_mfdataset(Sfilename, decode_times=True, combine='by_coords')#xr.open_dataset(Sfilename)
    L = xr.open_mfdataset(Lfilename, decode_times=True, combine='by_coords')#xr.open_dataset(Lfilename)
    U = xr.open_mfdataset(Ufilename, decode_times=True, combine='by_coords')#xr.open_dataset()
    
    # Convert temperature to degrees Celsius
    T['TREFHT'] = T['TREFHT'] - 273.15
    
    # Calculate ES and grdES
    ES = 611.0 * np.exp((17.27*T['TREFHT'])/(237.3+T['TREFHT']))
    grdES = (4098.0*ES/((237.3+T['TREFHT'])**2))
    
    # Calculate VP and VPD
    VP = RH['RH2M'] * ES / 100
    VPD = ES - VP
    
    # Calculate PSY
    CP = 1.00464 * 10**3  # J kg-1 K-1
    LV = 2.501 * 10**6  # J kg-1
    conv_fact = (24.0 * 60.0* 60.0 / LV)   # watts/m2 to mm/day
    PSY = (P['PSL'] * (CP/(0.622*LV)))
    
    #Calculate NET Radiation
    RN = (S['FSA'] - L['FIRA'])
    
    # Calculate U2
    U2 = U['U10'] * (np.log(128.0)/np.log(661.3))
    #Add the code to calculate the potential evapotranspiration:    
    t1 = grdES / (grdES + PSY)
    t2 = PSY / (grdES + PSY)
    
    Udum = U2
    Udum = 1.0
    t3 = 6430 * (Udum + 0.536 * U2)
    
    PET = (t1 * RN * conv_fact) + (t2 * t3 * VPD / LV)
    PET.attrs['long_name'] = 'Potential ET'
    PET.attrs['units'] = 'mm/day'
    print(PET)
    ! /bin/rm -f {outDir}/b.e21.BHIST_SSP370cmip6.f09_g17.CESM2-LE-{i}.clm2_h0.PET.185001-210012.nc
    fout1 = xr.Dataset()
    fout1['PET'] = PET
    fout1.to_netcdf(f"{outDir}/b.e21.BHIST_SSP370cmip6.f09_g17.CESM2-LE-{i}.clm2_h0.PET.185001-210012.nc")

<xarray.DataArray (time: 3012, lat: 2, lon: 288)>
dask.array<add, shape=(3012, 2, 288), dtype=float32, chunksize=(120, 2, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-02-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    long_name:  Potential ET
    units:      mm/day
<xarray.DataArray (time: 3012, lat: 2, lon: 288)>
dask.array<add, shape=(3012, 2, 288), dtype=float32, chunksize=(120, 2, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1850-02-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    long_name:  Potential ET
    units:      mm/day


In [14]:
print(T['TREFHT'].shape)

(3012, 192, 288)


In [ ]:
# Set the number of ensemble members
import glob
nEns = min(1, len(modName))

# Set the number of ensemble members
#nEns = len(modName)
# Loop through each ensemble member and calculate PET
for i in range(nEns):
    file_pattern1 = f"{inDirT}/*{modName[i]}*.h0.*.nc"
    file1 = glob.glob(file_pattern1)
    ft1 = xr.open_mfdataset(file1)
    TREFHT = ft1['TREFHT']
    #print(TREFHT)
    
    # Convert temperature to degree Celsius
    TREFHT = TREFHT - 273.15  # in degree Celsius
    ES = TREFHT
    ES = 611.0 * np.exp((17.27*TREFHT)/(237.3+TREFHT))  # N/m2
    grdES = ES 
    grdES = (4098.0*ES/((237.3+TREFHT)**2))  # unit N/m2 per degree Celsius
    
    # Load relative humidity data
    file_pattern2 = f"{inDirRH}/*{modName[i]}*.h0.*.nc"
    file2 = glob.glob(file_pattern2)
    ft2 = xr.open_mfdataset(file2)
    RH2M = ft2['RH2M']
    VP = RH2M
    VP = (RH2M * ES /100.0)
    VPD = RH2M
    VPD = (ES - VP)  # Vapor pressure deficit

    # Load sea level pressure data
    file_pattern3 = f"{inDirP}/*{modName[i]}*.h0.*.nc"
    file3 = glob.glob(file_pattern3)
    ft3 = xr.open_mfdataset(file3)
    PSL = ft3['PSL']
    CP = 1.00464 * 10**3  # J kg-1 K-1
    LV = 2.501 * 10**6  # J kg-1
    PSY = PSL
    PSY = (PSL * (CP/(0.622*LV)))  # N m-2 K-1
    conv_fact = (24.0 * 60.0* 60.0 / LV)  # watts/m2 to mm/day

    # Load FSA
    file_pattern4 = f"{inDirS}/*{modName[i]}*.h0.*.nc"
    file4 = glob.glob(file_pattern4)
    ft4 = xr.open_mfdataset(file4)
    FSA = ft4['FSA']
        
    # Load FIRA
    file_pattern5 = f"{inDirL}/*{modName[i]}*.h0.*.nc"
    file5 = glob.glob(file_pattern5)
    ft5 = xr.open_mfdataset(file5)
    FIRA = ft5['FIRA']
    
    RN = FSA
    RN = FSA - FIRA    # FIRA is postive upward (see page 63 in CLM4.5 tech doc)
        
    # Load U10
    file_pattern6 = f"{inDirU}/*{modName[i]}*.h0.*.nc"
    file6 = glob.glob(file_pattern6)
    ft6 = xr.open_mfdataset(file6)
    U10 = ft6['U10']
        
    U2 = U10
    U2 = U10 * (np.log(128.0)/np.log(661.3))  
    #Add the code to calculate the potential evapotranspiration:
    
    PET = RN
    PET.attrs['long_name'] = 'Potential ET'
    PET.attrs['units'] = 'mm/day'
    
    t1 = grdES / (grdES + PSY)
    t2 = PSY / (grdES + PSY)
    
    Udum = U2
    Udum = 1.0
    t3 = 6430 * (Udum + 0.536 * U2)
    
    PET[:] = (t1 * RN * conv_fact) + (t2 * t3 * VPD / LV)
    print(PET)

In [ ]:
PET = RN
    PET.attrs['long_name'] = 'Potential ET'
    PET.attrs['units'] = 'mm/day'
    
    t1 = grdES / (grdES + PSY)
    t2 = PSY / (grdES + PSY)
    
    Udum = U2
    Udum = 1.0
    t3 = 6430 * (Udum + 0.536 * U2)
    
    PET = (t1 * RN * conv_fact) + (t2 * t3 * VPD / LV)
PET = (grdES/(grdES+PSY)) * RN * conv_fact + (PSY/(grdES+PSY)) * 6430 * (1 + 0.536 * U2) * VPD / LV

In [ ]:
# Create output file with dimensions (time, lon, lat)
    fout1 = xr.Dataset(
        {'PET': xr.DataArray(PET, dims=['time', 'lon', 'lat'], coords={'time': PET.time, 'lon': PET.lon, 'lat': PET.lat})}
    )
    # Save output file as netCDF
    fout1.to_netcdf(f"{outDir}/b.e21.BHIST_SSP370cmip6.f09_g17.CESM2-LE-{modName[i]}.clm2_h0.PET.185001-210012.nc")

In [ ]:
# Set the number of ensemble members
import glob
nEns = nMod
import numpy as np

# Loop through each ensemble member and calculate PET
for i in modName:
    # Load temperature data
    file1 = glob.glob(inDirT + '/*' + i+'*.h0.*.nc')
    ft1 = xr.open_mfdataset(file1)
    TREFHT = ft1['TREFHT']
    #print(TREFHT)
    
    # Convert temperature to degree Celsius
    TREFHT = TREFHT - 273.15  # in degree Celsius
    ES = TREFHT
    ES = 611.0 * np.exp((17.27*TREFHT)/(237.3+TREFHT))  # N/m2
    grdES = ES 
    grdES = (4098.0*ES/((237.3+TREFHT)**2))  # unit N/m2 per degree Celsius
    
    # Load relative humidity data
    file2 = glob.glob(inDirRH + '/*' + i+'*.h0.*.nc')
    ft2 = xr.open_mfdataset(file2)
    RH2M = ft2['RH2M']
    VP = RH2M
    VP = (RH2M * ES /100.0)
    VPD = RH2M
    VPD = (ES - VP)  # Vapor pressure deficit

    # Load sea level pressure data
    file3 = glob.glob(inDirP + '/*' + i+'*.h0.*.nc')
    ft3 = xr.open_mfdataset(file3)
    PSL = ft3['PSL']
    CP = 1.00464 * 10**3  # J kg-1 K-1
    LV = 2.501 * 10**6  # J kg-1
    PSY = PSL
    PSY = (PSL * (CP/(0.622*LV)))  # N m-2 K-1
    conv_fact = (24.0 * 60.0* 60.0 / LV)  # watts/m2 to mm/day

    # Load FSA
    file4 = glob.glob(inDirS + '/*' + i+'*.h0.*.nc')
    ft4 = xr.open_mfdataset(file4)
    FSA = ft4['FSA']
        
    # Load FIRA
    file5 = glob.glob(inDirL + '/*' + i+'*.h0.*.nc')
    ft5 = xr.open_mfdataset(file5)
    FIRA = ft5['FIRA']
        
    # Load U10
    file6 = glob.glob(inDirU + '/*' + i+'*.h0.*.nc')
    ft6 = xr.open_mfdataset(file6)
    U10 = ft6['U10']
        
    U2 = U10
    U2 = U10 * (np.log(128.0)/np.log(661.3))

    
    #Add the code to calculate the potential evapotranspiration:
    RN = FSA - FIRA    # FIRA is postive upward (see page 63 in CLM4.5 tech doc)
    PET = (grdES/(grdES+PSY)) * RN * conv_fact + (PSY/(grdES+PSY)) * 6430 * (1 + 0.536 * U2) * VPD / LV
    
    PET = RN
    PET.attrs['long_name'] = 'Potential ET'
    PET.attrs['units'] = 'mm/day'
    
    t1 = grdES / (grdES + PSY)
    t2 = PSY / (grdES + PSY)
    
    Udum = U2
    Udum = 1.0
    t3 = 6430 * (Udum + 0.536 * U2)
    
    PET = (t1 * RN * conv_fact) + (t2 * t3 * VPD / LV)
    
    ! /bin/rm -f {outDir}/b.e21.BHIST_SSP370cmip6.f09_g17.CESM2-LE-{i}.clm2_h0.PET.185001-210012.nc
    fout1 = xr.Dataset()
    fout1['PET'] = PET
    fout1.to_netcdf(f"{outDir}/b.e21.BHIST_SSP370cmip6.f09_g17.CESM2-LE-{i}.clm2_h0.PET.185001-210012.nc")